<a href="https://colab.research.google.com/github/esheshka/chess_2/blob/main/chess_2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [179]:
# @title **Классы фигур**

from collections.abc import Iterator
from abc import ABC, abstractmethod
from typing import Literal, NamedTuple

class Piece:
    def __str__(self) -> str:
        return self.name

    @abstractmethod
    def move(self):
        raise NotImplementedError("This method should be overridden in subclasses.")

    @abstractmethod
    def verify(self, start: str, end: str, board) -> bool:
        raise NotImplementedError("This method should be overridden in subclasses.")

class Pawn(Piece):
    """Пешка."""
    def __init__(self, is_white):
        self.is_white = is_white
        self.name = 'P' if is_white else 'p'
        self.cost = 1

    def verify(self, start: str, end: str, board) -> bool:
        vector = get_vector(start=start, end=end)

        if not self.is_white:
            if start[-1] == "7" and vector == (0, -2):
                return True
            if (vector == (-1, -1) or vector == (1, -1)) and end in board:
                return True
            if vector == (0, -1):
                return True

        if self.is_white:
            if start[-1] == "2" and vector == (0, 2):
                return True
            if (vector == (1, 1) or vector == (-1, 1)) and end in board:
                return True
            if vector == (0, 1):
                return True

        return False

class Rook(Piece):
    """Ладья."""
    def __init__(self, is_white):
        self.is_white = is_white
        self.name = 'R' if is_white else 'r'
        self.cost = 5

    def verify(self, start: str, end: str, board) -> bool:
        vector = get_vector(start=start, end=end)
        return vector[0] == 0 or vector[1] == 0

class Knight(Piece):
    """Конь."""
    def __init__(self, is_white):
        self.is_white = is_white
        self.name = 'N' if is_white else 'n'
        self.cost = 3

    def verify(self, start: str, end: str, board) -> bool:
        vector = get_vector(start=start, end=end)
        vector_vals = {abs(v) for v in vector}
        return vector_vals == {1, 2}

class Bishop(Piece):
    """Слон."""
    def __init__(self, is_white):
        self.is_white = is_white
        self.name = 'B' if is_white else 'b'
        self.cost = 3

    def verify(self, start: str, end: str, board) -> bool:
        vector = get_vector(start=start, end=end)
        return abs(vector[0]) == abs(vector[1])

class Queen(Piece):
    """Королева."""
    def __init__(self, is_white):
        self.is_white = is_white
        self.name = 'Q' if is_white else 'q'
        self.cost = 9

    def verify(self, start: str, end: str, board) -> bool:
        vector = get_vector(start=start, end=end)
        x, y = vector
        return (abs(x) == abs(y)) or x == 0 or y == 0

class King(Piece):
    """Король."""
    def __init__(self, is_white):
        self.is_white = is_white
        self.name = 'K' if is_white else 'k'
        self.cost = 0

    def verify(self, start: str, end: str, board) -> bool:
        vector = get_vector(start=start, end=end)
        return abs(vector[0]) <= 1 and abs(vector[1]) <= 1

In [152]:
# @title **Класс доски**
COLUMN_NAMES = "ABCDEFGH"
SIZE_X = SIZE_Y = 8

class Board:
    def __init__(self):
        self.board = {}
        for column in range(SIZE_Y):
            self.board[COLUMN_NAMES[column] + "8"] = [Rook(False), Knight(False), Bishop(False), Queen(False), King(False), Bishop(False), Knight(False), Rook(False)][column]
            self.board[COLUMN_NAMES[column] + "7"] = Pawn(False)
            self.board[COLUMN_NAMES[column] + "1"] = [Rook(True),  Knight(True),  Bishop(True),  Queen(True),  King(True),  Bishop(True),  Knight(True),  Rook(True)][column]
            self.board[COLUMN_NAMES[column] + "2"] = Pawn(True)

        self.counter = 0

    def __str__(self) -> str:
        str_board = ""
        board = self.board

        str_board += "\n   A B C D E F G H\n\n"
        for row_i in reversed(range(1, SIZE_Y + 1)):
            str_board += f"{row_i}  "
            for column in COLUMN_NAMES:
                pos = f"{column}{row_i}"
                str_board += f"{board[pos]} " if pos in board else ". "
            str_board += f" {row_i}\n"
        str_board += f"\n   A B C D E F G H\n\n"

        return str_board

    def player_is_white(self):
        return self.counter % 2 == 0

    def num_white_pieces(self) -> int:
        ans = 0
        for piece in self.board.values():
            ans += 1 if piece.is_white else 0
        return ans

    def num_black_pieces(self) -> int:
        ans = 0
        for piece in self.board.values():
            ans += 1 if not piece.is_white else 0
        return ans

    def balance(self) -> int:
        ans = 0
        for piece in self.board.values():
            ans += piece.cost * (1 if piece.is_white else -1)
        return ans * -1

    def __contains__(self, piece: Piece) -> bool:
        for piece_on_board in self.board.values():
            if piece.name == piece_on_board.name:
                return True
        return False

    def __getitem__(self, pos) -> Piece:
        if pos in self.board:
            return self.board[pos]

In [67]:
from enum import Enum

ErrorType = Enum(
    "ErrorType",
    {
        "FORMAT": "Wrong input format",
        "INVALID_MOVE": "The piece cannot make the specified move",
    },
)

In [60]:
# @title **ТЕХНИЧЕСКАЯ ЧАСТЬ**
# *---------------------*#
# *--ТЕХНИЧЕСКАЯ ЧАСТЬ--*#
# *--НЕ РЕДАКТИРОВАТЬ!--*#
def check_task1():
    try:
        assert 'Piece' in sorted(globals()), "No root class 'Piece' in task"
        assert 'Board' in sorted(globals()), "No class 'Board' in task"
        assert len([c for c in sorted(globals()) if type(eval(c)) == type(Piece) and issubclass(eval(
            c), Piece) and eval(c).__name__ != 'Piece']) == 6, "Less then 6 child classes of class 'Piece'"
        assert 'num_white_pieces' in dir(Board) and 'num_black_pieces' in dir(
            Board), "No methods 'num_black_pieces' or 'num_white_pieces' in 'Board' class"
        assert 'balance' in dir(Board), "No method 'balance' in 'Board' class"
        assert '__contains__' in dir(
            Board), "No method '__contains__' in 'Board' class"
        assert 'Board' not in str(
            Board()), "No custom '__str__' method in class 'Board'"
        assert '__getitem__' in dir(
            Board), "No method '__getitem__' in 'Board' class"

    except AssertionError as e:
        print(
            f'Something went wrong with requirements written in task condition. Desciption: {e}')

check_task1()
# *---------------------*#

# **Логика передвижения**

In [181]:
def parse_move(inp: str) -> tuple[tuple[str, str] | None, ErrorType | None]:
    """Данная функция обрабатывает ввод хода: производит несколько проверок корректности и вызывает ранее объявленные функции проверок."""
    step = inp.upper().split("-")
    if len(step) != 2:
        return None, ErrorType.FORMAT

    pos_start, pos_end = step[0], step[1]
    if pos_start == pos_end:
        return None, ErrorType.INVALID_MOVE

    if is_incorrect_pos(pos_start) or is_incorrect_pos(pos_end):
        return None, ErrorType.FORMAT

    return ((pos_start, pos_end), None)


In [182]:
def validate_move(
    start: str, end: str, board: Board) -> tuple[bool, ErrorType | None]:
    fig = board[start]

    if fig is None:
        return False, ErrorType.INVALID_MOVE

    if not is_fig_friendly(fig, board.player_is_white()):
        return False, ErrorType.INVALID_MOVE

    if not fig.verify(start=start, end=end, board=board.board):
        return False, ErrorType.INVALID_MOVE

    is_knight = fig.name.lower() == "n"
    if not is_knight and has_obstacle(start=start, end=end, board=board.board):
        return False, ErrorType.INVALID_MOVE

    is_pawn = fig.name.lower() == "p"
    is_end_empty = end not in board.board
    end_fig = board[end]

    is_end_enemy = end_fig is not None and not is_fig_friendly(end_fig, board.player_is_white())

    v = get_vector(start, end)
    is_move_diag = v[0] != 0

    is_valid_pawn_move = is_end_empty or (is_move_diag and is_end_enemy)

    if is_pawn and not is_valid_pawn_move:
        return False, ErrorType.INVALID_MOVE

    if end in board.board:
        if is_fig_friendly(board[end], board.player_is_white()):
            return False, ErrorType.INVALID_MOVE

    return True, None

In [164]:
def has_obstacle(start: str, end: str, board: dict[str, str]) -> bool:
    """Функция проверяет наличие препятствий по ходу фигуры."""
    for path_func in (get_straight_path, get_diag_path):
        path = path_func(start, end)
        for temp_pos in path:
            if temp_pos is None:
                break

            if temp_pos in board:
                return True

    return False

In [163]:
def get_straight_path(start: str, end: str) -> Iterator[str | None]:
    v = get_vector(start, end)
    is_vector_straight = 0 in v

    if not is_vector_straight or start == end:
        return None

    start_int = pos_to_int(start)
    end_int = pos_to_int(end)

    zero_dim = v.index(0)
    non_zero_dim = 1 - zero_dim

    start_i, end_i = start_int[non_zero_dim], end_int[non_zero_dim]
    if start_i > end_i:
        start_i, end_i = end_i, start_i

    for i in range(start_i + 1, end_i):
        new_pos_int_ = list(start_int)
        new_pos_int_[non_zero_dim] = i
        new_pos = pos_from_int(tuple(new_pos_int_))  # type: ignore[arg-type]
        yield new_pos


def get_diag_path(start: str, end: str) -> Iterator[str | None]:
    v = get_vector(start, end)
    start_int = pos_to_int(start)
    end_int = pos_to_int(end)

    is_vector_diag = abs(v[0]) == abs(v[1])
    if not is_vector_diag or start == end:
        return None

    x_sgn = sgn(v[0])
    y_sgn = sgn(v[1])
    x_values = range(start_int[0] + x_sgn, end_int[0], x_sgn)
    y_values = range(start_int[1] + y_sgn, end_int[1], y_sgn)
    for i, j in zip(x_values, y_values):
        new_pos_int = (i, j)
        new_pos = pos_from_int(new_pos_int)
        yield new_pos

In [177]:
def sgn(number: int) -> Literal[-1, 0, 1]:
    """Вспомогательная функция, определяющая знак переданного числа."""
    if number > 0:
        return 1
    if number < 0:
        return -1
    return 0

In [81]:
def get_vector(start: str, end: str) -> tuple[int, int]:
    """Функция, возвращающая вектор перемещения фигуры."""
    start_v, end_v = (
        (COLUMN_NAMES.index(start[0]), int(start[-1])),
        (COLUMN_NAMES.index(end[0]), int(end[-1])),
    )
    return end_v[0] - start_v[0], end_v[-1] - start_v[-1]

In [82]:
def is_incorrect_pos(pos: str) -> bool:
    return (
        not len(pos) == 2
        or pos[-1] not in "12345678"
        or pos[0] not in COLUMN_NAMES
    )

In [83]:
def is_fig_friendly(fig: str, player_is_white: bool) -> bool:
    return fig.is_white == player_is_white

In [168]:
def pos_to_int(pos: str) -> tuple[int, int]:
    x, y = tuple(pos)
    return COLUMN_NAMES.index(x), int(y)

In [170]:
def pos_from_int(pos_int: tuple[int, int]) -> str:
    x, y = pos_int
    return f"{COLUMN_NAMES[x]}{y}"

In [172]:
def move_fig(start: str, end: str, board: Board) -> None:
    fig = board.board.pop(start)
    board.board[end] = fig

# **Запуск программы**

In [85]:
def log_error(err: ErrorType) -> None:
    print(f"Error. Type: {err.value}.")

In [184]:
def main() -> None:
    board = Board()

    while True:
        player_name = "white" if board.player_is_white() else "black"
        player_step_num = board.counter // 2 + 1
        cmd = input(f"{player_name} {player_step_num}:\n")

        if cmd == "draw":
            print(board)
            continue

        if cmd == "exit":
            return

        if cmd == "balance white":
            print(board.balance() * -1)
            continue
        if cmd == "balance black":
            print(board.balance())
            continue

        move, err = parse_move(cmd)
        if err is not None:
            log_error(err)
            continue
        assert move is not None
        pos_start, pos_end = move

        _, err = validate_move(
            start = pos_start,
            end = pos_end,
            board = board,
        )
        if err is not None:
            log_error(err)
            continue

        move_fig(start=pos_start, end=pos_end, board=board)

        board.counter += 1

if __name__ == "__main__":
    main()

white 1:
e2-e4
black 1:
f7-f5
white 2:
balance white
0
white 2:
balance black
0
white 2:
e4-f5
black 2:
balance white
1
black 2:
balance black
-1
black 2:
draw

   A B C D E F G H

8  r n b q k b n r  8
7  p p p p p . p p  7
6  . . . . . . . .  6
5  . . . . . P . .  5
4  . . . . . . . .  4
3  . . . . . . . .  3
2  P P P P . P P P  2
1  R N B Q K B N R  1

   A B C D E F G H


black 2:
exit
